In [30]:
#https://algotrading101.com/learn/yahoo-finance-api-guide/
import yfinance as yf
import yahoo_fin.stock_info as si
import pandas as pd
comp_ticker = yf.Ticker("EPD")


In [24]:
def my_valuation(ticker):
#Quote table
    key_attributes_quote=['PE Ratio (TTM)','Beta (5Y Monthly)','1y Target Est','EPS (TTM)','Previous Close','Market Cap']
    quote=si.get_quote_table(ticker)
    attr_results=[]
    for attribute in key_attributes_quote:
        attr_results.append(quote[attribute])
    attr_results_df=pd.DataFrame([attr_results],columns=key_attributes_quote)
    attr_results_df['Ticker']=ticker
    
#Stats table
    stats_table=si.get_stats(ticker)
    stats_table=stats_table.iloc[[22,24,46,47,48],:].transpose()
    stats_table.columns=stats_table.iloc[0,:]
    stats_table=stats_table.iloc[1,:]
    stats_table=pd.DataFrame(stats_table)
    stats_table=stats_table.transpose().reset_index(drop=True)
    
#Analysts recommendation
    lhs_url = 'https://query2.finance.yahoo.com/v10/finance/quoteSummary/'
    rhs_url = '?formatted=true&crumb=swg7qs5y9UP&lang=en-US&region=US&' \
              'modules=upgradeDowngradeHistory,recommendationTrend,' \
              'financialData,earningsHistory,earningsTrend,industryTrend&' \
              'corsDomain=finance.yahoo.com'
              
    url =  lhs_url + ticker + rhs_url
    r = requests.get(url)
    if not r.ok:
        recommendation = 6
    try:
        result = r.json()['quoteSummary']['result'][0]
        recommendation =result['financialData']['recommendationMean']['fmt']
    except:
        recommendation = 6
    stats_table['Recommendation']=recommendation
    
#Adding business summary
    comp_ticker=yf.Ticker(ticker)
    stats_table['Company Summary']=comp_ticker.info['longBusinessSummary']
    
#join
    horizontal_stack = pd.concat([stats_table, attr_results_df], axis=1)

#Changing columns order
    cols=horizontal_stack.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    horizontal_stack=horizontal_stack[cols]
    return horizontal_stack

In [28]:
#dow_list = si.tickers_dow()
my_list=['ADM','MDLZ','HUN','NEM','CVS','TSN']
for ticker in my_list:
    temp=my_valuation(ticker)
    instrument_results=pd.concat([instrument_results,temp],axis=0)

In [29]:
instrument_results.to_csv('dow_results.csv',index=False)